In [ ]:
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(temperature = 0.1)

class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query:str = Field(description="The query you will search for")

class StockMarketSymbolSearchTool(BaseTool):
    name: Type[str] = "StockMarketSymbolSearchTool"
    args_schema : Type[
        StockMarketSymbolSearchToolArgsSchema
        ] = StockMarketSymbolSearchToolArgsSchema
    description: Type[str] = """
        Use this tool to find the stock market symbol for a company.
        It takes a query as an argument.
        Example query: Stock Market Symbol for Apple Inc
    """

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)

agent = initialize_agent(
    llm=llm,
    verbose = True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors = True,
    tools=[
        StockMarketSymbolSearchTool(),
    ]
)

prompt = "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have."

agent.invoke(prompt)

/var/folders/cm/ycdd7_2d2fq3f98kcv14fznw0000gn/T/ipykernel_9070/2805861089.py:8: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature = 0.1)
/var/folders/cm/ycdd7_2d2fq3f98kcv14fznw0000gn/T/ipykernel_9070/2805861089.py:31: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Cloudflare'}`




TypeError: 'DuckDuckGoSearchAPIWrapper' object does not support the context manager protocol